In [1]:
import getpass
import os
from langchain_core.tools import tool
os.environ["OPENAI_API_KEY"] = getpass.getpass()



In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [6]:
from operator import itemgetter
from typing import Dict, List

from langchain_core.messages import AIMessage
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import tool


@tool
def count_emails(last_n_days: int) -> int:
    """Multiply two integers together."""
    return last_n_days * 2


@tool
def send_email(message: str, recipient: str) -> str:
    "Add two integers."
    return f"Successfully sent email to {recipient}."


tools = [count_emails, send_email]
llm_with_tools = llm.bind_tools(tools)


def call_tools(msg: AIMessage) -> List[Dict]:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


chain = llm_with_tools | call_tools
chain.invoke("how many emails did i get in the last 5 days?")

[{'name': 'count_emails',
  'args': {'last_n_days': 5},
  'id': 'call_eivvvx27e7Y6hoRIviW8I6ZQ',
  'type': 'tool_call',
  'output': 10}]

In [7]:
import json


def human_approval(msg: AIMessage) -> Runnable:
    tool_strs = "\n\n".join(
        json.dumps(tool_call, indent=2) for tool_call in msg.tool_calls
    )
    input_msg = (
        f"Do you approve of the following tool invocations\n\n{tool_strs}\n\n"
        "Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no."
    )
    resp = input(input_msg)
    if resp.lower() not in ("yes", "y"):
        raise ValueError(f"Tool invocations not approved:\n\n{tool_strs}")
    return msg

In [14]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are lover expert, you writing my words,
      I am Amro, you do not love, I love. okay ?
     Love is fire. """),
    ("user", "{input}")
])

In [17]:
chain = prompt | llm_with_tools | human_approval | call_tools
chain.invoke({"input": "send email to my love her email is karyna@lovesamro.com?"})

[{'name': 'send_email',
  'args': {'message': 'Dear Karyna, Love is fire. - Amro',
   'recipient': 'karyna@lovesamro.com'},
  'id': 'call_nURW0lFqXtt40RFr3UhFYYTk',
  'type': 'tool_call',
  'output': 'Successfully sent email to karyna@lovesamro.com.'}]